# NFL Baseline
- create target_df (distance in tracking_df is lower than threshold=3)
https://www.kaggle.com/code/stgkrtua/nfl-creatatraindataset-targetdf
- create dataset save frames in target_df
https://www.kaggle.com/code/stgkrtua/nfl-createdataset-saveframes
- check saved images
https://www.kaggle.com/code/stgkrtua/nfl-checkdataset-plotsavedimage

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import GroupKFold

# loss metrics
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score

import cudf

import mlflow
# import wandb
# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [2]:
CFG = {
        "kaggle" : False,
        "DEBUG" : False,
        # model config
        "model_name" : "swin_s3_tiny_224",
        "out_features" : 20,
        "inp_channels": 3,
        "num_img_feature" : 5,
        "pretrained" : True,
        
        "roll_sum_window_size" : 10,
        # "features" : ['x_position_1', 'y_position_1', 'x_position_2', 'y_position_2', 
        #               'speed_1', 'distance_1', 'direction_1', 'orientation_1','acceleration_1', 'sa_1', 
        #               'speed_2', 'distance_2', 'direction_2', 'orientation_2', 'acceleration_2', 'sa_2',
        #               'speed_diff', 'distance_diff', 'direction_diff', 'orientation_diff','acceleration_diff', 'sa_diff', # diff-feature
        #               'players_dis', 'is_ground'
        #               'x_move_distance_1', 'y_move_distance_1', 'x_move_distance_2', 'y_move_distance_2',
        #               'x_position_diff_rollsum','y_position_diff_rollsum', 
        #               'speed_diff_rollsum','distance_diff_rollsum', 'direction_diff_rollsum','orientation_diff_rollsum', 
        #               'acceleration_diff_rollsum', 'sa_diff_rollsum','players_dis_rollsum', 
        #               ],

        # learning config
        "n_epoch" : 5,
        "n_folds": 3,
        "train_folds" : [0,1,2],
        "lr" : 1e-4,
        "T_max" : 10,
        "min_lr" : 1e-8,
        "weight_decay" : 1e-6,

        # etc
        "print_freq" : 1000,
        "random_seed" : 21,

        # data config    
        "img_size" : (224, 224),
        "batch_size" : 32,
        "num_workers" : 2,
        "masksize_helmet_ratio" : 4, # helmetサイズにこの係数をかけたサイズだけ色を残して後は黒塗りする
        "TRAIN_VIDEO_NUM" : 100,
        "VALID_VIDEO_NUM" : 10,
        "sample_num" : -1, 

        "EXP_CATEGORY" : "exps_cudf",
        "EXP_NAME" : "expC002_pretrain_PorGplayers_swins3tiny",
}

if CFG["DEBUG"]:
    CFG["EXP_CATEGORY"] = "DEBUG"
    CFG["EXP_NAME"] = "DEBUG"
    CFG["n_epoch"] = 2
    CFG["sample_num"] = 1000
    CFG["batch_size"] = 32
    CFG["train_folds"] : [0,1]


CFG["INPUT_DIR"] = "/workspace/input"
CFG["OUTPUT_DIR"] = "/workspace/output"
CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_baseline_helmets.csv")
CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_player_tracking.csv")
CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_video_metadata.csv")
CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_labels.csv")
CFG["SAVED_CONTACT_CSV"] = os.path.join(CFG["INPUT_DIR"], "Saved_contact_frames.csv")
CFG["CONTACT_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "contact_images")
CFG["MODEL_DIR"] = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"] ,"model")
    
if not CFG["kaggle"] and not CFG["DEBUG"]:
    os.mkdir(os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"]))
    os.mkdir(CFG["MODEL_DIR"])


In [3]:
if CFG["kaggle"]:
    os.environ["WANDB_SILENT"] = "true"
    WANDB_CONFIG = {'competition': 'NFL', '_wandb_kernel': 'taro'}
    # Secrets
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb")

    !wandb login $secret_value_0
    #! TODO : logger settings
    wandb.init(project=WANDB_CONFIG["competition"], config=CFG, group=CFG["EXP_CATEGORY"], name=CFG["EXP_NAME"])

else:
    mlflow.set_tracking_uri("/workspace/mlruns")
    experiment = mlflow.get_experiment_by_name(CFG["EXP_CATEGORY"])
    if experiment is None:  # 当該Experiment存在しないとき、新たに作成
        experiment_id = mlflow.create_experiment(name=CFG["EXP_CATEGORY"])
    else: # 当該Experiment存在するとき、IDを取得
        experiment_id = experiment.experiment_id

# Utils

In [4]:
def logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, score, threshold, tn_best, fp_best, fn_best, tp_best, auc_score):
    if CFG["kaggle"]:
        wandb.log({"loss avg":{f"train/fold{fold}": train_loss_avg,
                                f"valid/fold{fold}": valid_loss_avg}}, step=epoch)
        wandb.log({"Metircs" : {f"score/fold{fold}":score,
                                f"score threshold/fold{fold}":threshold,
                                f"tn/fold{fold}":tn_best,
                                f"fp/fold{fold}":fp_best,
                                f"fn/fold{fold}":fn_best,
                                f"tp/fold{fold}":tp_best,
                                f"auc/fold{fold}":auc_score,
                               }}, step=epoch)
    else:
        mlflow.log_metric(f"fold{fold} train loss avg", train_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} valid loss avg", valid_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} score", score, step=epoch)
        mlflow.log_metric(f"fold{fold} score threshold", threshold, step=epoch)
        mlflow.log_metric(f"fold{fold} tn", tn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fp", fp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fn", fn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} tp", tp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} auc", auc_score, step=epoch)

In [5]:
def seed_everything(seed=CFG["random_seed"]):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Dataset Functions

In [7]:
def add_feature_cols(df_, FEATURE_COLS,remove_col_list):
    additional_cols = list(df_.columns)
    additional_cols = [col for col in additional_cols if not col in remove_col_list]
    FEATURE_COLS.extend(additional_cols)
    return FEATURE_COLS

## target df func

In [8]:
def create_trackmerged_ftr(target_df, FEATURE_COLS,
                           diff_cols = ['x_position', 'y_position', 'speed', 'distance',
                                        'direction', 'orientation', 'acceleration', 'sa']):
    # players distance features
    target_df["players_dis"] = np.sqrt((target_df["x_position_1"] - target_df["x_position_2"])**2 
                                       + (target_df["y_position_1"] - target_df["y_position_2"])**2)
    # GがNanになる。これは0にする
    target_df["players_dis"] = target_df["players_dis"].fillna(0)
    FEATURE_COLS.append("players_dis")
    
    # players distance sum(in shift range time : default(-6~6 frames not step))
    players_distance_sum = 0
    for idx in range(-6,6,1):
        players_distance_sum += np.sqrt((target_df[f"x_position_shift{idx}_1"] - target_df[f"x_position_shift{idx}_2"])**2 
                                       + (target_df[f"y_position_shift{idx}_1"] - target_df[f"y_position_shift{idx}_2"])**2)
    target_df["players_distance_sum"] = players_distance_sum
    # GがNanになる。これは0にする
    target_df["players_distance_sum"] = target_df["players_distance_sum"].fillna(0)
    FEATURE_COLS.append("players_distance_sum")

    # players each axis distance sum(in shift range time : default(-6~6 frames not step))
    for axis in ["x", "y"]:
        axis_distance_1 = 0
        axis_distance_2 = 0
        for idx in range(-6, 5, 1):
            axis_distance_1 += abs(target_df[f"{axis}_position_shift{idx}_1"] - target_df[f"{axis}_position_shift{idx+1}_1"])
            axis_distance_2 += abs(target_df[f"{axis}_position_shift{idx}_2"] - target_df[f"{axis}_position_shift{idx+1}_2"])
        target_df[f"{axis}_move_distance_1"] = axis_distance_1
        target_df[f"{axis}_move_distance_2"] = axis_distance_2
        FEATURE_COLS.extend([f"{axis}_move_distance_1", f"{axis}_move_distance_2"])
    
    # players difference ftr (in each step)
    for col in diff_cols:
        colname = f"{col}_diff"
        target_df[colname] = abs(target_df[f"{col}_1"] - target_df[f"{col}_2"])
        FEATURE_COLS.append(colname)
    
    return target_df, FEATURE_COLS

In [9]:
def create_roll_ftr(target_df, FEATURE_COLS,
                    roll_cols = ['players_dis', 'x_position_diff', 'y_position_diff', 
                                  'speed_diff', 'distance_diff','direction_diff',
                                  'orientation_diff', 'acceleration_diff', 'sa_diff']):
    print("original length", len(target_df))
    key_cols = ["game_play", "nfl_player_id_1", "nfl_player_id_2"]
    roll_df = target_df[roll_cols+key_cols].copy()
    roll_df["key"] = roll_df["game_play"] + "_" + roll_df["nfl_player_id_1"].astype("str") + "_" + roll_df["nfl_player_id_2"].astype("str")

    group_roll_df = roll_df.groupby("key").rolling(CFG["roll_sum_window_size"])[roll_cols].sum().fillna(-999).sort_index()
    for col in roll_cols:
        group_roll_df = group_roll_df.rename(columns={col:col+"_rollsum"})
        FEATURE_COLS.append(col+"_rollsum")

    if not torch.cuda.is_available():
        target_df = pd.concat([target_df, group_roll_df], axis=1).sort_index()
    else:
        target_df = cudf.concat([target_df, group_roll_df], axis=1).sort_index()
    print("after length", len(target_df))
    return target_df, FEATURE_COLS

In [10]:
def create_helmetmerged_ftr(target_df, FEATURE_COLS):
    # helmet center distance feature
    for view in ["Endzone", "Sideline"]:
        for p_id in ["1", "2"]: 
            # get helmet center
            col_name = f"{view[0]}_Wcenter_{p_id}"
            pos_col, size_col =  f"{view[0]}_left_{p_id}", f"{view[0]}_width_{p_id}"
            target_df[col_name] = target_df[pos_col] + (target_df[size_col]//2)
            col_name = f"{view[0]}_Hcenter_{p_id}"
            pos_col, size_col =  f"{view[0]}_top_{p_id}", f"{view[0]}_height_{p_id}"
            target_df[col_name] = target_df[pos_col] + (target_df[size_col]//2)
        
        # helmet center distance
        target_df[f"{view[0]}_helmet_dis"] = np.sqrt((target_df[f"{view[0]}_Wcenter_1"] - target_df[f"{view[0]}_Wcenter_2"])**2 
                                             + (target_df[f"{view[0]}_Hcenter_1"] - target_df[f"{view[0]}_Hcenter_2"])**2)
        # GがNanになるので0にしておく
        target_df[f"{view[0]}_helmet_dis"] = target_df[f"{view[0]}_helmet_dis"].fillna(0)
        FEATURE_COLS.append(f"{view[0]}_helmet_dis")
    
    # helmet cols fillna(0) after get helmet distance 
    helmet_cols = ['E_left_1', 'E_width_1', 'E_top_1', 'E_height_1',
                   'E_left_2', 'E_width_2', 'E_top_2', 'E_height_2', 
                   'S_left_1','S_width_1', 'S_top_1', 'S_height_1', 
                   'S_left_2', 'S_width_2', 'S_top_2', 'S_height_2']
    target_df[helmet_cols] = target_df[helmet_cols].fillna(0)

    return target_df, FEATURE_COLS

In [11]:
def get_categorical_ftr(target_df, FEATURE_COLS):
    target_df["is_ground"] = (target_df["nfl_player_id_2"] == "G").astype(np.int64)
    target_df["players_dis"].mask((target_df["is_ground"]==1), 0, inplace=True)
    
    target_df["nfl_player_id_2"] = target_df["nfl_player_id_2"].replace("G", "99999").astype(np.int64) # when inference this is after cnn pred
    target_df["is_helmet"] = 1 - ((target_df["E_width_1"]==0) & (target_df["E_width_2"]==0)
                                  & (target_df["S_width_1"]==0) & (target_df["S_width_2"]==0)).astype(np.int64)
    
    # set team 
    target_df["team_1"] = (target_df["team_1"] == "home").astype(np.int64)
    target_df["team_2"] = (target_df["team_2"] == "home").astype(np.int64)
    
    FEATURE_COLS.extend(["is_ground", "is_helmet"])
    return target_df, FEATURE_COLS

## tracking df func

In [12]:
def get_tracking_shift(tracking_df, shift_cols=["x_position","y_position"], shift_nums=range(-6,6,1)):
    # get shift key
    tracking_df["game_player"] = tracking_df["game_play"].str.cat(tracking_df['nfl_player_id'].astype(str), sep='_')
    tracking_df["frame_add"] = (tracking_df['step']/10*59.94+5*59.94).astype('int')+5000 #全部0以上の方が並べやすい
    tracking_df["frame_key"] = tracking_df["frame_add"].astype(str).str.zfill(5)
    tracking_df["shift_key"] = tracking_df["game_player"].str.cat(tracking_df["frame_key"].astype(str), sep='_')
    tracking_df = tracking_df.sort_values("shift_key").reset_index(drop=True)
#     display(tracking_df[["shift_key", "game_player", "step", "x_position","y_position"]])
    SHIFT_COLS = []
    for col in shift_cols:
        for num in shift_nums:
            colname = f"{col}_shift{num}"
            tracking_df[colname] = tracking_df[col].shift(num)
            SHIFT_COLS.append(colname)
    tracking_df = tracking_df.drop(["game_player", "frame_add","frame_key", "shift_key"], axis=1)
    return tracking_df, SHIFT_COLS

In [13]:
def target_merge_tracking(target_df, tracking_df, FEATURE_COLS, SHIFT_COLS,
                          TRACKING_COLS = ["game_play", "nfl_player_id", "step", 
                                           "x_position", "y_position", "datetime",
                                           "speed","distance","direction","orientation",
                                           "acceleration","sa", "team", "jersey_number"] ):
    print("original length", len(target_df))
    # set merge-key (game_step_player_1, 2) to merge tracking_df
    target_df["game_step"] = target_df['game_play'].str.cat(target_df['step'].astype(str), sep='_')
    target_df["game_step_player_1"] = target_df['game_step'].str.cat(target_df['nfl_player_id_1'].astype(str), sep='_')
    target_df["game_step_player_2"] = target_df['game_step'].str.cat(target_df['nfl_player_id_2'].astype(str), sep='_')

    # merge-key
    TRACKING_COLS.extend(SHIFT_COLS)
    tracking_df = tracking_df[TRACKING_COLS]
    tracking_df["frame"] = (tracking_df['step']/10*59.94+5*59.94).astype('int')+1
    tracking_df["game_step"] = tracking_df['game_play'].str.cat(tracking_df['step'].astype(str), sep='_')
    tracking_df["game_step_player"] = tracking_df['game_step'].str.cat(tracking_df['nfl_player_id'].astype(str), sep='_')
    tracking_df = tracking_df.drop(["game_step", "game_play", "step", "nfl_player_id"], axis=1)
    
    for player_id in [1,2]:
        tracking_player = tracking_df.copy()
        tracking_player.rename(columns={"game_step_player":f"game_step_player_{player_id}"}, inplace=True)
        rename_cols = [col for col in tracking_player.columns if col != f"game_step_player_{player_id}"]
        tracking_player = tracking_player.rename(columns={rename_col: f"{rename_col}_{player_id}" for rename_col in rename_cols})
        target_df = target_df.merge(tracking_player, on=[f"game_step_player_{player_id}"], how="left")
        # add features col
        FEATURE_COLS = add_feature_cols(tracking_player, FEATURE_COLS,
                                        [f"game_step_player_{player_id}", f"frame_{player_id}", f"datetime_{player_id}"])
    target_df["frame"] = target_df["frame_1"]
    FEATURE_COLS.append("frame")
    
    target_df = target_df.drop(["frame_1", "frame_2", "game_step_player_1", "game_step_player_2",
                                "datetime_1", "datetime_2"], axis=1)
#     print(target_df.columns)
    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS

## helmet df func

In [14]:
def target_merge_helmet(target_df, helmet_df, FEATURE_COLS):
    print("original length", len(target_df))
    # set merge-key (game_frame_player_1,2) to merge helmet_df
    target_df["game_frame"] = target_df['game_play'].str.cat(target_df['frame'].astype(str), sep='_')
    target_df["game_frame_player_1"] = target_df['game_frame'].str.cat(target_df['nfl_player_id_1'], sep='_')
    target_df["game_frame_player_2"] = target_df['game_frame'].str.cat(target_df['nfl_player_id_2'], sep='_')
    # set merge key
    helmet_df["game_frame"] = helmet_df['game_play'].str.cat(helmet_df['frame'].astype(str), sep='_')
    helmet_df["game_frame_player"] = helmet_df['game_frame'].str.cat(helmet_df['nfl_player_id'].astype(str), sep='_')
    
    # merge target df & helmet_df
    player_views = [[1, "Endzone"],[2, "Endzone"], [1, "Sideline"],[2, "Sideline"]]
    for player_id, view in player_views:
        helmet_view = helmet_df[helmet_df["view"]==view]
        helmet_view = helmet_view[["game_frame_player", "left", "width", "top", "height"]]
        helmet_view.rename(columns={"game_frame_player":f"game_frame_player_{player_id}"}, inplace=True)
        rename_cols = helmet_view.columns[1:]
        helmet_view = helmet_view.rename(columns={rename_col: f"{view[0]}_{rename_col}_{player_id}" for rename_col in rename_cols})
        if not torch.cuda.is_available():
            target_df = pd.merge(target_df, helmet_view, on=f"game_frame_player_{player_id}", how="left")
        else:
            target_df = target_df.merge(helmet_view, on=[f"game_frame_player_{player_id}"], how="left")  
        # add features col
        FEATURE_COLS = add_feature_cols(helmet_view, FEATURE_COLS, [f"game_frame_player_{player_id}"])

    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS

---

# Load Target

In [15]:
if not torch.cuda.is_available():
    target_df = pd.read_csv(CFG["TRAIN_LABEL_CSV"], parse_dates=["datetime"])    
else:
    target_df = cudf.read_csv(CFG["TRAIN_LABEL_CSV"], parse_dates=["datetime"])
FEATURE_COLS = ["nfl_player_id_1", "nfl_player_id_2", "step"]
display(target_df)

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11 03:01:48.100,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11 03:01:48.100,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11 03:01:48.100,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11 03:01:48.100,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11 03:01:48.100,0,38590,47944,0
...,...,...,...,...,...,...,...
4721613,58582_003121_91_48220_G,58582_003121,2021-10-12 02:42:29.100,91,48220,G,0
4721614,58582_003121_91_47906_G,58582_003121,2021-10-12 02:42:29.100,91,47906,G,0
4721615,58582_003121_91_38557_G,58582_003121,2021-10-12 02:42:29.100,91,38557,G,0
4721616,58582_003121_91_47872_G,58582_003121,2021-10-12 02:42:29.100,91,47872,G,0


# Merge tracking_df

In [16]:
if not torch.cuda.is_available():
    tracking_df = pd.read_csv(CFG["TRAIN_TRACKING_CSV"], parse_dates=["datetime"])
else:
    tracking_df = cudf.read_csv(CFG["TRAIN_TRACKING_CSV"], parse_dates=["datetime"])

In [17]:
tracking_df, SHIFT_COLS = get_tracking_shift(tracking_df)
target_df, FEATURE_COLS = target_merge_tracking(target_df, tracking_df, FEATURE_COLS, SHIFT_COLS)
del tracking_df

original length 4721618
77
original length 4721618


In [18]:
target_df, FEATURE_COLS = create_trackmerged_ftr(target_df, FEATURE_COLS)
target_df, FEATURE_COLS = create_roll_ftr(target_df, FEATURE_COLS)

original length 4721618
after length 4721618


In [19]:
target_df = target_df[target_df["players_dis"] <= 2].reset_index(drop=True)
print(len(target_df))

660560


# Merge helmet df

In [20]:
if not torch.cuda.is_available():
    helmet_df = pd.read_csv(CFG["TRAIN_HELMET_CSV"])
else:
    helmet_df = cudf.read_csv(CFG["TRAIN_HELMET_CSV"])

target_df, FEATURE_COLS = target_merge_helmet(target_df, helmet_df, FEATURE_COLS)
del helmet_df

original length 660560
119
original length 660560


In [21]:
target_df, FEATURE_COLS = create_helmetmerged_ftr(target_df, FEATURE_COLS)
target_df, FEATURE_COLS = get_categorical_ftr(target_df, FEATURE_COLS)
print(len(target_df))

660560


# Reduce Data

In [22]:
target_df["is_E_helmet"] = 1 -  ((target_df["E_width_1"]==0) & (target_df["E_width_2"]==0)).astype(int)
target_df["is_S_helmet"] = 1 -  ((target_df["S_width_1"]==0) & (target_df["S_width_2"]==0)).astype(int)
target_df["is_helmet"] = (target_df["is_E_helmet"] + target_df["is_S_helmet"] > 0).astype(int)
target_df = target_df[target_df["is_ground"]==0]

target_df_E = target_df[target_df["is_E_helmet"]==1]
target_df_E["EorS"] = "Endzone"

target_df_S = target_df[target_df["is_S_helmet"]==1]
target_df_S["EorS"] = "Sideline"

target_df = cudf.concat([target_df_E, target_df_S], axis=0).reset_index(drop=True)

if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)

target_df = target_df.to_pandas()

print(len(target_df))
print(len(target_df["game_play"].unique()))
display(target_df["contact"].value_counts())

448936
240


0    357894
1     91042
Name: contact, dtype: int64

# Augmentation

In [23]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.3, 0.3), p=0.5),
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

valid_transform = A.Compose([
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

# Dataset

In [24]:
class NFLDataset(Dataset):
    def __init__(self, target_df, transform=None):
        self.target_df = target_df
        self.transform = transform

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        # read frame image
        game_play = target_info.game_play
        frame = target_info.frame
        view = target_info.EorS
        contact_id = target_info.contact_id
        contact_fileid = f"{contact_id}_{view}.jpg"
        contact_filename = os.path.join(CFG["CONTACT_IMG_DIR"], contact_fileid)
        img = cv2.imread(contact_filename)
        if img is None:
            img = np.zeros((224, 224, 3))
            img = np.transpose(img, (2, 0, 1)).astype(np.float)
            img = torch.tensor(img, dtype=torch.float)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform:
                img = self.transform(image=img)["image"]
            else:
                img = img / 255. # convert to 0-1
                img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
        input_img = img

        target = torch.tensor(target, dtype=torch.float)

        return input_img, target

# Model

In [25]:
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG["model_name"],
        out_features = CFG["num_img_feature"],
        inp_channels= CFG["inp_channels"],
        pretrained = CFG["pretrained"]
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes=out_features)
        self.fc = nn.Linear(CFG["num_img_feature"], 1)

    def forward(self, img):
        img_emb = self.model(img)
        output = self.fc(img_emb)
        return output, img_emb

# train fn

In [26]:
def train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (images, targets) in enumerate(train_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)      
        preds, _ = model(images)
        
        loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"]) 
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        del preds, images, targets
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [27]:
def valid_fn(model, valid_loader, criterion):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []
    img_embs = []

    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (images, targets) in enumerate(valid_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)
        with torch.no_grad():
            preds, img_emb = model(images)
            loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"])
        batch_time.update(time.time() - end)

        img_emb = img_emb.detach().cpu().numpy()
        img_embs.extend(img_emb)

        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        test_preds.extend(preds)
        test_targets.extend(targets)
        # score = matthews_corrcoef(preds, targets)
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(valid_loader)-1):
            print('\t EVAL: [{0}/{1}] '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                    batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                    remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                ))
        del preds, images, targets
        gc.collect()
        torch.cuda.empty_cache()
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    return test_targets, test_preds, img_embs, losses.avg

# Train loop

In [28]:
def training_loop(target_df):
    # set model & learning fn
    oof_df = pd.DataFrame()
    kf = GroupKFold(n_splits=CFG["n_folds"])
    for fold, (idx_train, idx_valid) in enumerate(kf.split(target_df, target_df["contact_id"], target_df["game_play"])):
        print("---")
        print(f"fold {fold} start training...")
        model = NFLNet()
        model = model.to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"], amsgrad=False)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG["T_max"], eta_min=CFG["min_lr"], last_epoch=-1)

        if not fold in CFG["train_folds"]:
            print(f"fold{fold} is skip")
            continue
        # separate train/valid data 
        train_df = target_df.iloc[idx_train]
        valid_df = target_df.iloc[idx_valid]
        print("train target contact")
        print(train_df["contact"].value_counts())
        print("valid target contact")
        print(valid_df["contact"].value_counts())
        # separate train/valid data 
        train_dataset = NFLDataset(train_df, train_transform)
        valid_dataset = NFLDataset(valid_df, valid_transform)
        train_loader = DataLoader(train_dataset,batch_size=CFG["batch_size"], shuffle = True,
                                    num_workers = CFG["num_workers"], pin_memory = True)
        valid_loader = DataLoader(valid_dataset,batch_size=CFG["batch_size"], shuffle = False,
                                    num_workers = CFG["num_workers"], pin_memory = True)

        # training
        best_score = -np.inf
        best_auc = -np.inf
        start_time, end = time.time(), time.time()
        for epoch in range(1, CFG["n_epoch"] + 1):
            print(f'\t === epoch: {epoch}: training ===')
            train_loss_avg = train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler)
            valid_targets, valid_preds, valid_embs, valid_loss_avg = valid_fn(model, valid_loader, criterion)

            valid_score = -np.inf
            valid_threshold = 0
            tn_best, fp_best, fn_best, tp_best = 0, 0, 0, 0
            for idx in range(1, 10, 1):
                thr = idx*0.1
                valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
                valid_binary_preds = (np.array(valid_preds) > thr).astype(np.int32)
                score_tmp = matthews_corrcoef(valid_targets, valid_binary_preds)
                cm = confusion_matrix(valid_targets, valid_binary_preds)
                tn, fp, fn, tp = cm.flatten()
                if score_tmp > valid_score:
                    valid_score = score_tmp 
                    valid_threshold = thr
                    tn_best, fp_best, fn_best, tp_best = tn, fp, fn, tp
            elapsed = (time.time() - start_time)/60
            auc_score = roc_auc_score(valid_targets, valid_preds)
            print(f'\t epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}')
            print(f'\t score:{valid_score:.4f}(th={valid_threshold}) AUC = {auc_score:.4f}=> time:{elapsed:.2f} min')
            scheduler.step()
            # validationスコアがbestを更新したらモデルを保存する
            if valid_score > best_score:
                best_score = valid_score
                model_name = CFG["model_name"]
                torch.save(model.state_dict(), f'{CFG["MODEL_DIR"]}/{model_name}_fold{fold}.pth')
                print(f'\t Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
                contact_id = valid_df["contact_id"].values
                _oof_df = pd.DataFrame({
                    "contact_id" : contact_id,
                    "pred" : valid_preds,
                    "contact" : valid_targets,
                    "fold" : fold,
                })
                img_emb_colname = [f"img_emb_{idx}" for idx in range(CFG["num_img_feature"])]
                img_emb_df = pd.DataFrame(valid_embs, columns=img_emb_colname)
                _oof_df = pd.concat([_oof_df, img_emb_df], axis=1)
            
            logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, valid_score, valid_threshold, tn_best, fp_best, fn_best, tp_best, auc_score)

        del train_loader, train_dataset, valid_loader, valid_dataset
        oof_df = pd.concat([oof_df, _oof_df], axis = 0)
        del _oof_df
        gc.collect()
        torch.cuda.empty_cache()
    return oof_df

In [29]:
if CFG["kaggle"]:
    oof_df = training_loop(target_df)
    wandb.finish()
else:
    with mlflow.start_run(experiment_id=experiment_id, run_name=CFG["EXP_NAME"]) as run:
        mlflow.log_dict(CFG, "configuration.yaml")
        mlflow.log_param("positive data num", len(target_df[target_df["contact"]==1]))
        mlflow.log_param("negative data num", len(target_df[target_df["contact"]==0]))
        oof_df = training_loop(target_df)

---
fold 0 start training...
train target contact
0    238096
1     61178
Name: contact, dtype: int64
valid target contact
0    119798
1     29864
Name: contact, dtype: int64
	 === epoch: 1: training ===
	 Epoch: [1][0/9353] Elapsed 0m 0s (remain 98m 4s) Loss: 0.7431(0.7431) 
	 Epoch: [1][1000/9353] Elapsed 1m 57s (remain 16m 21s) Loss: 0.4135(0.4044) 
	 Epoch: [1][2000/9353] Elapsed 3m 55s (remain 14m 25s) Loss: 0.2761(0.3836) 
	 Epoch: [1][3000/9353] Elapsed 5m 52s (remain 12m 26s) Loss: 0.4039(0.3761) 
	 Epoch: [1][4000/9353] Elapsed 7m 49s (remain 10m 27s) Loss: 0.3272(0.3698) 
	 Epoch: [1][5000/9353] Elapsed 9m 46s (remain 8m 30s) Loss: 0.2947(0.3649) 
	 Epoch: [1][6000/9353] Elapsed 11m 43s (remain 6m 32s) Loss: 0.6000(0.3615) 
	 Epoch: [1][7000/9353] Elapsed 13m 39s (remain 4m 35s) Loss: 0.2993(0.3586) 
	 Epoch: [1][8000/9353] Elapsed 15m 35s (remain 2m 38s) Loss: 0.3417(0.3561) 
	 Epoch: [1][9000/9353] Elapsed 17m 33s (remain 0m 41s) Loss: 0.4065(0.3534) 
	 Epoch: [1][9352/9353

# Save oof_df

In [30]:
display(oof_df)
if CFG["kaggle"]:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)
else:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)

,contact_id,pred,contact,fold,img_emb_0,img_emb_1,img_emb_2,img_emb_3,img_emb_4
0,58204_003581_9_38683_47865,0.003769,0,0,5.348165,3.900079,6.240396,-6.564895,-14.779208
1,58198_003771_24_33107_40077,0.813972,1,0,-1.185989,-1.062252,-1.589648,1.676396,3.273607
2,58198_003771_24_33107_44870,0.002711,0,0,5.728329,4.097891,6.588621,-6.971368,-15.610251
3,58172_003247_31_41937_45345,0.832085,1,0,-1.251008,-1.217480,-1.688563,1.823383,3.607216
4,58176_002844_27_43439_52426,0.067114,0,0,2.613202,1.900698,3.218463,-3.217485,-7.073386
...,...,...,...,...,...,...,...,...,...
149622,58536_002108_30_35470_43484,0.909062,1,2,-2.591925,0.511374,-0.036288,3.468956,-1.668542
149623,58536_002108_30_42352_47792,0.060929,1,2,3.181828,-0.972838,0.015880,-4.127928,2.308740
149624,58536_002108_30_42352_42831,0.045734,0,2,3.524462,-1.079319,0.009826,-4.593266,2.528523
149625,58536_002108_30_42352_45012,0.028326,0,2,4.118843,-1.184555,0.033949,-5.334674,2.921659
